<div style="display: flex; justify-content: space-between;">
<a style="flex: 1; text-align: left;" href="./3_1_MLR_bus_weather.ipynb">← Previous: 3.1 Model Building with Weather</a>
<a style="flex: 1; text-align: right;" href="./4_Conclusions.ipynb">Next: Conclusions →</a>
</div>

### 3.2 Model Building with Traffic
---


<div style="text-align: justify; line-height: 2">

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Having observed that weather conditions do not have a significant impact on the end delay of STM buses, we will attempt to build a model that also includes traffic data to see if it can improve the model's predictive ability. The new model will include the same weather and bus data as the previous unfiltered model. This data will be combined with the traffic data from the City of Montreal4 . [See master + traffic dataset description](../Data/Dataset_descriptions.md). A sample of the data is shown below.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|     | line | delay_start | start_time | temp_start | rain_amt | snow_amt | delay_end | total_traffic |
| :-: | ---: | ----------: | ---------: | ---------: | -------: | -------: | --------: | ------------: |
|  0  |  467 |           0 |        405 |        7.1 |        0 |        0 |         6 |          3674 |
|  1  |  467 |           1 |        437 |        6.9 |        0 |        0 |         5 |          3116 |
|  2  |  467 |           1 |        468 |        6.9 |        0 |        0 |         6 |          3984 |
|  3  |  467 |          -1 |        496 |        6.5 |        0 |        0 |         6 |          2655 |
|  4  |  467 |           0 |        527 |        6.5 |        0 |        0 |         6 |          2273 |

|     | T_NBLT | T_NBT | T_NBRT | T_SBLT | T_SBT | T_SBRT | T_EBLT | T_EBT | T_EBRT | T_WBLT | T_WBT | T_WBRT |
| :-: | -----: | ----: | -----: | -----: | ----: | -----: | -----: | ----: | -----: | -----: | ----: | -----: |
|  0  |      0 |  1236 |     23 |      4 |  1270 |    155 |    165 |   157 |     68 |      2 |   478 |    116 |
|  1  |      0 |  1031 |     56 |      0 |   927 |    103 |    130 |   238 |     45 |      0 |   473 |    113 |
|  2  |      0 |  1330 |     44 |      0 |  1167 |    170 |    191 |   312 |     89 |      3 |   567 |    111 |
|  3  |      0 |   853 |     35 |      0 |   884 |    113 |    116 |   171 |     86 |      1 |   321 |     75 |
|  4  |      0 |   762 |     32 |      0 |   774 |     81 |     96 |   164 |     47 |      1 |   253 |     63 |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.1: Sample of the data used for model building with traffic.</p>

</td>
</tr>
</table>

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|   Column Name   |                                    Description                                    |
| :-------------: | :-------------------------------------------------------------------------------: |
|     *line*      |                                  Bus line number                                  |
|  *delay_start*  |                    Delay at the start of the trip (in minutes)                    |
|  *start_time*   |         Time of day at the start of the trip (in minutes after midnight)          |
|  *temp_start*   |             Temperature at the start of the trip (in degrees Celsius)             |
|   *rain_amt*    |                   Rain amount at the start of the trip (in cm)                    |
|   *snow_amt*    |                   Snow amount at the start of the trip (in cm)                    |
|   *delay_end*   | Delay at the end of the trip (in minutes, negative values indicate early arrival) |
| *total_traffic* |               Total traffic during the trip (in number of vehicles)               |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.2: Description of the columns in the data set used for model building with traffic.</p>

</td>
</tr>
</table>

The coded columns represent the number of vehicles with a certain direction and turn at all the intersections during the trip. The first two letters represent the direction bound (N, S, E, W), and the last letters represent the turn (LT = left turn, T = through, RT = right turn). For example, T_NBLT represents the number of vehicles going through the intersection in the northbound direction and turning left. The total traffic is the sum of all the coded columns.

The correlation heatmap for the predictors is shown in Figure 3.2.1.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">
  
<p style="text-align: center;"><img src="../Images/3_2_1_corr_mat_3.png" alt="Figure 3.2.1" /></p>

<p style="text-align: center;"><strong><em>Figure 3.2.1: Correlation heatmap for the data set used for model building with traffic.</p>

</td>
</tr>
</table>

The correlation heatmap shows that the traffic data has some significant correlations with the delay at the end of the trip. The strongest correlations, excluding coded traffic columns, are with the total traffic (~0.17) and the delay at the start of the trip (~0.11). The correlations are still relatively weak, so we do not expect the traffic data to have definitive predictive ability. Note that the traffic at each direction and turn have greater correlations with the delay at the end of the trip than the total traffic. This is expected since bus routes have a fixed direction, so the traffic in the opposite direction should not affect the bus. However, the high correlations between all the coded traffic columns indicate that there is a lot of multicollinearity in these portions of the data. We will opt to use the total traffic as the sole traffic predictor to avoid multicollinearity issues. To further observe the relationship between the predictors and the response, we plot the scatter plots of the response against the predictors in Figure 3.2.2.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

<p style="text-align: center;"><img src="../Images/3_2_2_scatter_plots_3.png" alt="Figure 3.2.2" /></p>

<p style="font-weight: bold; font-style: italic; text-align: center;">Figure 3.2.2: Scatter plots of the response against the predictors for the data set used for model building with traffic.</p>

</td>
</tr>
</table>

As with the previous models, the scatter plots show that the relationship between the delay at the end of the trip and the delay at the start of the trip is non-linear, while the weather predictors have no apparent relationship with the response. The scatter plot of the total traffic shows a positive non-linear relationship between the delay at the end of the trip and the total traffic, which is more apparent by observing the scatter plots of the individual directions and turns. The shape of the distribution is close to exponential. To observe the relationship more clearly, we will separate the data by bus line to make individual models for each bus line, if found to be significant. However, we will first build a model with all the data to observe the results of the t-tests. The model summary is shown in Table 3.2.3.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

| OLS Regression Results |                  |                         |           |
| :--------------------- | ---------------: | :---------------------- | --------: |
| **Dep. Variable:**     |        delay_end | **R-squared:**          |     0.050 |
| **Model:**             |              OLS | **Adj. R-squared:**     |     0.049 |
| **Method:**            |    Least Squares | **F-statistic:**        |     44.87 |
| **Date:**              | Mon, 25 Dec 2023 | **Prob (F-statistic):** |  8.69e-54 |
| **Time:**              |         14:42:26 | **Log-Likelihood:**     |   -17009. |
| **No. Observations:**  |             5134 | **AIC:**                | 3.403e+04 |
| **Df Residuals:**      |             5127 | **BIC:**                | 3.408e+04 |
| **Df Model:**          |                6 |                         |           |
| **Covariance Type:**   |        nonrobust |                         |           |

|                   |    coef |  std err |      t | P>&vert;t&vert; | [0.025 | 0.975] |
| :---------------- | ------: | -------: | -----: | --------------: | -----: | -----: |
| **delay_start**   |  0.0417 |    0.008 |  5.312 |               0 |  0.026 |  0.057 |
| **start_time**    |  0.0019 |        0 |  5.827 |               0 |  0.001 |  0.003 |
| **temp_start**    |  0.0433 |    0.012 |  3.549 |               0 |  0.019 |  0.067 |
| **rain_amt**      |  0.6417 |    0.129 |   4.96 |               0 |  0.388 |  0.895 |
| **snow_amt**      |  11.364 |   10.207 |  1.113 |           0.266 | -8.647 | 31.375 |
| **total_traffic** |  0.0004 | 2.75e-05 | 12.874 |               0 |      0 |      0 |
| **const**         | -1.6028 |    0.294 |  -5.45 |               0 | -2.179 | -1.026 |

|                    |          |                       |         |
| :----------------- | -------: | :-------------------- | ------: |
| **Omnibus:**       | 1201.053 | **Durbin-Watson:**    |   1.368 |
| **Prob(Omnibus):** |        0 | **Jarque-Bera (JB):** | 8628.43 |
| **Skew:**          |    0.927 | **Prob(JB):**         |       0 |
| **Kurtosis:**      |    9.075 | **Cond. No.**         |  474000 |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.3: Model summary for the model built with all lines and traffic.</p>

</td>
</tr>
</table>

The model summary shows that snow amount is not significant with a p-value of 0.266, so we will remove it from the model. The model summary for the new model is shown in Table 3.2.4.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

| OLS Regression Results |                  |                         |           |
| :--------------------- | ---------------: | :---------------------- | --------: |
| **Dep. Variable:**     |        delay_end | **R-squared:**          |     0.050 |
| **Model:**             |              OLS | **Adj. R-squared:**     |     0.049 |
| **Method:**            |    Least Squares | **F-statistic:**        |     53.60 |
| **Date:**              | Mon, 25 Dec 2023 | **Prob (F-statistic):** |  2.07e-54 |
| **Time:**              |         14:58:36 | **Log-Likelihood:**     |   -17010. |
| **No. Observations:**  |             5134 | **AIC:**                | 3.403e+04 |
| **Df Residuals:**      |             5128 | **BIC:**                | 3.407e+04 |
| **Df Model:**          |                5 |                         |           |
| **Covariance Type:**   |        nonrobust |                         |           |

|                   |    coef |  std err |      t | P>&vert;t&vert; | [0.025 | 0.975] |
| :---------------- | ------: | -------: | -----: | --------------: | -----: | -----: |
| **delay_start**   |  0.0416 |    0.008 |   5.31 |               0 |  0.026 |  0.057 |
| **start_time**    |  0.0019 |        0 |  5.831 |               0 |  0.001 |  0.003 |
| **temp_start**    |  0.0409 |    0.012 |  3.406 |           0.001 |  0.017 |  0.064 |
| **rain_amt**      |  0.6421 |    0.129 |  4.963 |               0 |  0.389 |  0.896 |
| **total_traffic** |  0.0004 | 2.75e-05 | 12.868 |               0 |      0 |      0 |
| **const**         | -1.5621 |    0.292 | -5.353 |               0 | -2.134 |  -0.99 |

|                    |          |                       |         |
| :----------------- | -------: | :-------------------- | ------: |
| **Omnibus:**       | 1200.710 | **Durbin-Watson:**    |   1.368 |
| **Prob(Omnibus):** |        0 | **Jarque-Bera (JB):** | 8621.65 |
| **Skew:**          |    0.927 | **Prob(JB):**         |       0 |
| **Kurtosis:**      |    9.072 | **Cond. No.**         |   13600 |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.4: Model summary for the model built with all lines and traffic without snow amount.</p>

</td>
</tr>
</table>

The adjusted R-squared value did not change, so the snow amount was not significant. This R-squared value is the same as the previous unfiltered model. However, we will first build a model for each bus line as traffic may have a different effect on each bus line. To ensure that the model is not overfitting, we will only build models for the bus lines with more than 300 data points. The number of data points for each bus line is shown in Table 3.2.5.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|    Line |    Count |
| ------: | -------: |
| **467** |  **395** |
|  **67** | **1218** |
|     480 |      171 |
| **121** | **1592** |
|      80 |      103 |
| **100** |  **789** |
| **460** |  **481** |
|     139 |       77 |
| **439** |  **308** |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.5: Number of data points for each bus line.</p>

</td>
</tr>
</table>

The bus lines with more than 500 data points are highlighted in bold. For simplicity, the whole model summary will not be shown for each bus line. The adjusted R-squared and the p-values for the predictors are shown in Table 3.2.6.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|                      |   467.0 |     67.0 |    121.0 |   100.0 |   460.0 |   439.0 |
| :------------------- | ------: | -------: | -------: | ------: | ------: | ------: |
| delay_start          |   0.026 |    0.002 |    0.123 |       0 |       0 |       0 |
| start_time           |   0.261 |        0 |        0 |       0 |       0 |       0 |
| temp_start           |   0.005 |    0.014 |    0.068 |       0 |       0 |   0.359 |
| rain_amt             |   0.892 |    0.444 |    0.867 |   0.143 |   0.557 |   0.792 |
| total_traffic        |   0.004 |        0 |        0 |       0 |       0 |       0 |
| const                |   0.382 |    0.255 |    0.001 |    0.01 |   0.001 |   0.098 |
| *Adjusted R-squared* |  *0.04* |  *0.029* |  *0.082* | *0.335* | *0.318* | *0.266* |
| *datapoints*         | *395.0* | *1218.0* | *1592.0* | *789.0* | *481.0* | *308.0* |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.6: Adjusted R-squared and p-values for the predictors for each bus line.</p>

</td>
</tr>
</table>

The adjusted

































</div>
&nbsp;  
&nbsp;  

---

<div style="display: flex; justify-content: space-between;">
<a style="flex: 1; text-align: left;" href="./3_1_MLR_bus_weather.ipynb">← Previous: 3.1 Model Building with Weather</a>
<span style="flex: 1; text-align: center;">3.2 Model Building with Traffic</span>
<a style="flex: 1; text-align: right;" href="./4_Conclusions.ipynb">Next: Conclusions →</a>
</div>
